In [2]:
import pandas as pd
import joblib

# Modelle laden
regressor = joblib.load("random_forest_model.joblib")

# falls Hyperparameteroptimierung durchgeführt wurde
# regressor = joblib.load("best_random_forest_model.joblib")
n_of_datapoints_to_predict = 20
data = pd.read_csv("dataset.csv", nrows=n_of_datapoints_to_predict)

# Berechnung von sick_time
min_timestamps = data.groupby(["c_van17", "c_serial_number"])[
    "msg_timestamp"
].transform("min")
data["sick_time"] = data["msg_timestamp"] - min_timestamps

# Eingabeparameter
columns_to_keep = [
    "InvSampTime",
    "RadV_50per",
    "OfsDevTight",
    "InvSampTurn",
    "AmpDevStrict",
    "RadV_25per",
    "RawRadius",
    "AmpSyncCheck",
    "msg_timestamp",
    "OfsCos200",
    "OfsCos100",
    "ClCos300",
    "OfsSin200",
    "ClSin300",
    "time_to_fail",
    "mileage",
    "ClSin150",
    "ClCos150",
    "OfsSin100",
    "EcuLifeT",
    "sick_time",
]

# DataFrame auf die gewünschten Spalten reduzieren
test_df = data[columns_to_keep]

# Zielvariable entfernen
non_numerical_features = ["time_to_fail"]
numerical_features = test_df.drop(columns=non_numerical_features)

# Vorhersage mit dem Regressor
regressor_prediction = regressor.predict(test_df[numerical_features.columns])

# Ergebnisse anzeigen und mit den tatsächlichen Werten vergleichen
results = []
for i in range(len(regressor_prediction)):
    results.append(
        {
            "Difference in days": float(
                (regressor_prediction[i] - test_df.iloc[i]["time_to_fail"])
                / 1000
                / 3600
                / 24
            ), # positiv: Model hat zu spät vorhergesagt; negative: Model hat zu früh vorhergesagt
            "Actual": float(test_df.iloc[i]["time_to_fail"] / 1000 / 3600 / 24),
            "Predicted": float(regressor_prediction[i] / 1000 / 3600 / 24),
        }  
    )


# Ergebnisse ausgeben
for result in results:
    print(result)

{'Difference in days': 0.586906424629631, 'Actual': 4.164534155092593, 'Predicted': 4.751440579722224}
{'Difference in days': 0.586906447314816, 'Actual': 4.164534131944444, 'Predicted': 4.751440579259261}
{'Difference in days': 0.9715966718055556, 'Actual': 3.1042580324074067, 'Predicted': 4.075854704212963}
{'Difference in days': 0.9715966949537037, 'Actual': 3.1042580092592593, 'Predicted': 4.075854704212963}
{'Difference in days': 1.145655277285053, 'Actual': 2.101806747685185, 'Predicted': 3.2474620249702384}
{'Difference in days': 1.1456552888591272, 'Actual': 2.101806736111111, 'Predicted': 3.2474620249702384}
{'Difference in days': 0.8091028597646607, 'Actual': 1.069527199074074, 'Predicted': 1.8786300588387348}
{'Difference in days': 0.8091028713387348, 'Actual': 1.0695271875, 'Predicted': 1.8786300588387348}
{'Difference in days': 2.532084258294753, 'Actual': 0.0, 'Predicted': 2.532084258294753}
{'Difference in days': -13.450282302314816, 'Actual': 102.33687738425925, 'Predic